# 📈 Análisis avanzado – Cohortes y economía unitaria
Sesión 2 - Ciencia de Datos y Producto

### 🎯 Objetivo de la sesión
Aplicar métodos de análisis de cohortes para entender el comportamiento de los usuarios a lo largo del tiempo y evaluar la rentabilidad del negocio usando economía unitaria (LTV, CAC).


### 1️⃣ ¿Qué es una cohorte?

📘 **Concepto:** Grupo de usuarios con una acción común en un mismo periodo.  
💡 **Intuición:** Permite comparar cómo evoluciona cada grupo a lo largo del tiempo.  
🛠️ **Aplicaciones:** Analizar retención, impacto de cambios y estacionalidad.

### 2️⃣ Tasa de retención y cancelación

📘 **Concepto:**  
- Retención = usuarios activos / usuarios iniciales  
- Cancelación = abandono respecto al mes anterior

💡 **Intuición:** Cuántos usuarios siguen usando el producto.  
🛠️ **Aplicaciones:** Medir fidelidad, detectar caídas, monitorear salud del producto.

### 3️⃣ Cohortes por comportamiento

📘 **Concepto:** Agrupar por acciones específicas, no solo por fecha.  
💡 **Intuición:** Segmentar por tipo de uso, no solo por cuándo llegaron.  
🛠️ **Aplicaciones:** Evaluar features, campañas o hábitos de uso.

### 4️⃣ Visualización con mapas de calor

📘 **Concepto:** Tabla de cohortes donde los colores indican intensidad (retención o ingreso).  
💡 **Intuición:** Fácil detección visual de tendencias.  
🛠️ **Aplicaciones:** Comunicación de hallazgos, análisis de retención.

### 5️⃣ LTV (Lifetime Value)

📘 **Concepto:** Ingreso total que aporta un cliente durante su ciclo de vida.  
💡 **Intuición:** ¿Cuánto valor genera un usuario?  
🛠️ **Aplicaciones:** Planeación de marketing, modelos de pricing.

### 6️⃣ CAC (Customer Acquisition Cost)

📘 **Concepto:** Costo promedio de adquirir un cliente.  
💡 **Intuición:** ¿Cuánto nos cuesta cada usuario nuevo?  
🛠️ **Aplicaciones:** Evaluación de campañas y rentabilidad.

### 7️⃣ Relación LTV / CAC

📘 **Concepto:** Medida clave de rentabilidad.  
💡 **Intuición:** Si LTV > CAC, hay ganancia. Ideal: LTV/CAC ≈ 3.  
🛠️ **Aplicaciones:** Decisiones de inversión, viabilidad del modelo.

### 🧪 Simulación de datos para cohortes y economía unitaria

In [ ]:
import pandas as pd
import numpy as np

# Simulación de datos
np.random.seed(42)
n_users = 50
user_ids = range(1001, 1001 + n_users)
registro = pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-06-30", freq="D"), n_users))
compras = [registro[i] + pd.to_timedelta(np.random.randint(0, 120), unit="D") for i in range(n_users)]
montos = np.random.randint(20, 200, n_users)
cac = np.random.randint(5, 30, n_users)

df = pd.DataFrame({
    "user_id": user_ids,
    "fecha_registro": registro,
    "fecha_compra": compras,
    "monto": montos,
    "costo_adquisicion": cac
})

df["mes_cohorte"] = df["fecha_registro"].dt.to_period("M")
df["mes_compra"] = df["fecha_compra"].dt.to_period("M")
df.head()


### 📊 Retención de usuarios por cohorte

In [ ]:
cohort_table = df.groupby(["mes_cohorte", "mes_compra"])["user_id"].nunique().unstack().fillna(0)
cohort_size = cohort_table.iloc[:, 0]
retencion = cohort_table.divide(cohort_size, axis=0)
retencion.style.format("{:.1%}")


### 🌡️ Visualización con mapa de calor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(retencion, annot=True, fmt=".0%", cmap="Blues", linewidths=1, linecolor='gray')
plt.title("Mapa de calor de retención por cohorte")
plt.ylabel("Cohorte de registro")
plt.xlabel("Mes de compra")
plt.tight_layout()
plt.show()


### 💰 Cálculo de LTV por cohorte

In [ ]:
ltv_table = df.groupby(["mes_cohorte", "mes_compra"])["monto"].sum().unstack().fillna(0).cumsum(axis=1)
ltv_table.style.format("${:.0f}")


### 🧮 CAC promedio y comparación con LTV

In [ ]:
cac_promedio = df["costo_adquisicion"].mean()
ltv_promedio = df.groupby("user_id")["monto"].sum().mean()
ratio = ltv_promedio / cac_promedio

print(f"CAC promedio: ${cac_promedio:.2f}")
print(f"LTV promedio: ${ltv_promedio:.2f}")
print(f"Relación LTV / CAC: {ratio:.2f}")
